# 1. if two users have less than 5 movies in common, skip this users

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
from MovieRecommend import train_test_seen_unseen, weight_calculator,pred_specific_movie_score, pred_series_movie_scores, raw_to_pred

In [3]:
rating = pd.read_csv('rating.csv', index_col=0)

/Users/coffeyliu/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
rating.head()

,movieId,rating,timestamp
userId,,,
1,2,3.5,2005-04-02 23:53:47
1,29,3.5,2005-04-02 23:31:16
1,32,3.5,2005-04-02 23:33:39
1,47,3.5,2005-04-02 23:32:07
1,50,3.5,2005-04-02 23:29:40


In [5]:
flattened = pd.DataFrame(rating.to_records())

In [6]:
flattened.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
sample = flattened.head(100000)

In [8]:
#get user_movie_table
user_movie_table = pd.pivot_table(sample, index=['userId'], columns=['movieId'])

In [9]:
len(user_movie_table) #==> we have 702 users

702

In [10]:
user_movie_table.head()

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          NaN    3.5    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2          NaN    NaN    4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    4.0   
5          NaN    3.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ...                                                                   \
movieId  ...   117511 117590 118354 118696 118900 118997 119141 125916 128488   
userId   ...                                                                    
1        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 128594  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  

[5 rows x 8227 columns]

In [11]:
#building training, test datasets
train, test, watched_index, unseen_index = train_test_seen_unseen(user_movie_table, 1)

In [12]:
index, score = pred_series_movie_scores(user_movie_table, 1, test, 20)

In [13]:
pred = pd.DataFrame()
pred['movieId'] = index
pred['raw_score'] = score
pred['pred_score'] = [raw_to_pred(x) for x in score]
pred = pred.set_index(keys='movieId')
# pred

In [14]:
true_score = pd.DataFrame(pd.DataFrame(user_movie_table.ix[1, test]).to_records()).drop('level_0', axis=1)
true_score.columns = ['movieId', 'true_score']
true_score = true_score.set_index('movieId')
# true_score

/Users/coffeyliu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [17]:
result = pred.join(true_score, on='movieId')
result

,raw_score,pred_score,true_score
movieId,,,
318,4.507784,4.5,4.0
2762,4.506975,4.5,4.0
260,4.234806,4.0,4.0
541,4.215892,4.0,4.0
1097,4.157003,4.0,4.0
7438,4.050004,4.0,4.0
6754,4.031269,4.0,4.0
2872,3.996057,4.0,4.0
1246,3.986073,4.0,3.5


In [19]:
mse_user1 = mean_squared_error(result['true_score'], result['pred_score'])
print(mse_user1)

0.08333333333333333


# it looks good :D